# Agent with Long-Time Memory
* We will build an Agent that will help us to **manage a ToDo list**.
* It will decide:
    * **when to save items** to our ToDo list.
    * **to save either a user profile or a collection of ToDo items**.
* In addition to semantic memory (user facts), it will also have **procedural memory**.
    * Remember, the procedural memory is the system prompt. This will allow the user to set preferences for creating ToDo items.

In [ ]:
#pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ.get("Open_ai_key_here")

Lets install Langchain here...



In [ ]:
#!pip install langchain-openai

In [ ]:
from langchain_openai import ChatOpenA

chatModel35 = ChatOpenAI(model="gpt-3.5-turbo-0125")
chatModel4o = ChatOpenAI(model = "gpt-4o")



Using TrustCall here....

In [ ]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    conent: str = Field(description= "The main content of the memory. For example:" \
    "User expressed interest in learning about French language...")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

In [ ]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

## Everytime you make instance of this class, you make a new list of tools used...

class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        ### This is here will collect information about the tools used in the extractor...
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                    
                )
spy = Spy()

##Initialize the model here....
model = ChatOpenAI(model="gpt-4o", temperature=0)

## create the extractor here....

trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice= "Memory",
    enable_inserts=True,
)

### Add the spy as a listener to the extractor....

trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)



## Running Trustcall without "listener" to monitor the workflow tool call...

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


## Instruction to the model to extract the memories from the user input...
instruction = """Extract memmories from the following converstion 'Mac'..."""

### Converstion here....
conversation =[HumanMessage(content="Hi I'm Chris"),
               AIMessage(content= "Nice to meet you 'Jackass'"),
               HumanMessage(content="Yesterday I visited the museum of History")]

### Using the regular extractor here without the listener....
result = trustcall_extractor.invoke("messages": [SystemMessage(content= instruction)] + conversation})




In [ ]:
### Messages that contain tool calls....

for m in result["messages"]:
    m.pretty_print()

    

